# Exploring CHB-MIT dataset

## Reading edf files

In [14]:
patient_id = 1
DATASET_PATH = os.path.join(os.getcwd(), 'data', 'chb01-summary.txt') #ONLY FOR TESTING
patient_id_str = str(patient_id).zfill(2)

all_file_path = glob(f'chb{patient_id_str}/*.edf')
print(len(all_file_path))

4


In [15]:
raw = mne.io.read_raw_edf(all_file_path[0])
print(raw.get_channel_types())
print(raw.info)
print(raw.ch_names)
print(raw.get_data().shape)

Extracting EDF parameters from d:\Codes\machine_learning\SCC0276-Machine-Learning-preprocessing\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\Calvin\AppData\Local\Temp\ipykernel_19436\2924192598.py:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(all_file_path[0])


['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg']
<Info | 7 non-empty values
 bads: []
 ch_names: FP1-F7, F7-T7, T7-P7, P7-O1, FP1-F3, F3-C3, C3-P3, P3-O1, ...
 chs: 23 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2076-11-06 11:42:54 UTC
 nchan: 23
 projs: []
 sfreq: 256.0 Hz
>
['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']
(23, 921600)


In [16]:
def read_data_edf(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5, h_freq=45)
    epochs = mne.make_fixed_length_epochs(data, duration=10, overlap=1)
    epochs_array = epochs.get_data()
    return epochs, epochs_array
# def read_data_seizures(file_path):
#     ref_file = open(file_path, 'r')
#     array=epochs.get_data()
#     return array

def read_edf_to_raw(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    raw.set_eeg_reference()
    raw.filter(l_freq=0.5, h_freq=45)
    return raw

In [17]:
%%capture

# Shape ()
epochs_data_list = [read_data_edf(i)[1] for i in all_file_path]
epochs_list = [read_data_edf(i)[0] for i in all_file_path]


In [18]:
epochs_data_array = np.array(epochs_data_list)
epochs_data_array.shape

(4, 399, 23, 2560)

In [20]:
# merge the files on one array 
# shape: (N_files, 399 epochs, 23 channels, 2560 values) ->  (N_files*399 epochs, 23 channels, 2560 values)
data_array = np.vstack(epochs_data_array)
data_array.shape

(1596, 23, 2560)

## Extract features

In [21]:
########! Features extraction !#####
from scipy import stats
def mean(x):
    return np.mean(x, axis=-1)
def std(x):
    return np.std(x, axis=-1)
def ptp(x):
    return np.ptp(x, axis=-1)
def var(x):
    return np.var(x, axis=-1)
def minim(x):
    return np.min(x, axis=-1)
def maxim(x):
    return np.max(x, axis=-1)
def argminim(x):
    return np.argmin(x, axis=-1)
def argmaxim(x):
    return np.argmax(x, axis=-1)
def rms(x):
    return np.sqrt(np.mean(x**2, axis=-1))
def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1)
def skewness(x):
    return stats.skew(x, axis=-1)
def kurtosis(x):
    return stats.kurtosis(x, axis=-1)
def concatenate_features(x):
    return np.concatenate((mean(x), std(x), ptp(x), var(x), minim(x), maxim(x), argmaxim(x), 
                           argminim(x), rms(x), abs_diff_signal(x), skewness(x), kurtosis(x)), axis=-1)

In [22]:
features = []
for d in data_array:
    features.append(concatenate_features(d))
    
features_array = np.array(features)
features_array.shape

(1596, 276)

## Visualize features per channel and per epoch

In [23]:
# get features of specific channel
def features_by_channel(features_array):
    features_channel = []
    MAX_CHANNELS = 23
    for i in range(MAX_CHANNELS):
        array = features_array[:, i*12:(i+1)*12];
        features_channel.append(pd.DataFrame(array, columns=['Mean', 'Std', 'Ptp', 'Var', 'Minimo', 'Maxim', 'Arg Max', 'Arg Min', 'RMS', 'ABS DIFF', 'skewness', 'kurtosis']))
    return features_channel

features_channel = features_by_channel(features_array)
print('Info for channel 23: ')

# Shape of features_channel=(23, 1596, 12)
features_channel[22].shape

Info for channel 23: 


(1596, 12)

# Preprocessing and labelling data

In [45]:
# patient_folder = Path.cwd()/'chbmit'/'chb{}'.format(str(patient_id).zfill(2))
# DATASET_PATH = 
DATASET_PATH = os.getcwd() #ONLY FOR TESTING

patient_id = 1
patient_id_str = str(patient_id).zfill(2)

patient_folder = os.path.join(DATASET_PATH, f'chb{patient_id_str}')
print(f"Serching for patient {patient_id_str} files at {patient_folder} ...")
patient_files = glob(os.path.join(patient_folder, f'*.edf'))
print(f"Found {len(patient_files)} files")
summary_path = os.path.join(patient_folder, f'chb{patient_id_str}-summary.txt')

Serching for patient 01 files at d:\Codes\machine_learning\SCC0276-Machine-Learning-preprocessing\chb01 ...
Found 9 files


In [46]:
## READ SUMMARY (TODO: automate this)
seizures_dict = {"chb01_03": [[2996, 3036]],
                "chb01_04": [[1467, 1494]],
                "chb01_15": [[1732, 1772]],
                "chb01_16": [[1015, 1066]],
                "chb01_18": [[1720, 1810]],
                "chb01_21": [[327, 420]],
                "chb01_26": [[1862, 1963]],
                
                "chb02_16": [[130, 212]],

                "chb05_06": [[417, 532]], 
                "chb05_13": [[1086, 1196]],
                "chb05_16": [[2317, 2413]], 
                "chb05_17": [[2451, 2571]],
                "chb05_22": [[2348, 2465]],
                
                "chb08_02": [[2670, 2841]], 
                "chb08_05": [[2856, 3046]],
                "chb08_11": [[2988, 3211]], 
                "chb08_13": [[2417, 2577]],
                "chb08_21": [[2083, 2347]]}

curr_time = 0
epoch_time = 10
overlap = 5

# Divide into epochs
labels = []
for file_path in patient_files:
    # get filename: chbxx_xx.edf
    filename = os.path.split(file_path)[1]

    # remove .edf staying only chbxx_xx
    filename = os.path.splitext(filename)[0]
    
    # read raw
    raw = read_edf_to_raw(file_path)

    curr_time = 0
    while curr_time <= max(raw.times) + 0.01 - epoch_time:  # max(raw.times) = 3600
        epoch_features = []
        start_time = curr_time 
        if start_time < 0.:
            start_time = 0.
        end_time = curr_time + epoch_time
        start, stop = raw.time_as_index([start_time, end_time])
        temp = raw[:, start:stop][0]

        # start time as ID
        # features.append(start_time)

        # features
        epoch_features.extend(concatenate_features(temp))

        # seizure flag for y
        aux = []
        if filename in seizures_dict:  # if file has seizure
            for seizure in seizures_dict[filename]:
                if start_time > seizure[0] and start_time < seizure[1]:
                    aux.append(1)
                elif start_time + epoch_time > seizure[0] and start_time + epoch_time < seizure[1]:
                    aux.append(1)
                else:
                    aux.append(0)
        else:    
            aux.append(0)

        if 1 in aux:
            epoch_features.extend([1])
        else:
            epoch_features.extend([0])

        labels.append(epoch_features)

        curr_time = curr_time + epoch_time - overlap  
        print("Section ", str(len(labels)), "; start: ", start, " ; stop: ", stop)

Extracting EDF parameters from d:\Codes\machine_learning\SCC0276-Machine-Learning-preprocessing\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


C:\Users\Calvin\AppData\Local\Temp\ipykernel_19436\4000768790.py:14: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_path, preload=True)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)

Section  1 ; start:  0  ; stop:  2560
Section  2 ; start:  1280  ; stop:  3840
Section  3 ; start:  2560  ; stop:  5120
Section  4 ; start:  3840  ; stop:  6400
Section  5 ; start:  5120  ; stop:  7680
Section  6 ; start:  6400  ; stop:  8960
Section  7 ; start:  7680  ; stop:  10240
Section  8 ; start:  8960  ; stop:  11

C:\Users\Calvin\AppData\Local\Temp\ipykernel_19436\4000768790.py:14: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_path, preload=True)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)

Section  720 ; start:  0  ; stop:  2560
Section  721 ; start:  1280  ; stop:  3840
Section  722 ; start:  2560  ; stop:  5120
Section  723 ; start:  3840  ; stop:  6400
Section  724 ; start:  5120  ; stop:  7680
Section  725 ; start:  6400  ; stop:  8960
Section  726 ; start:  7680  ; stop:  10240
Section  727 ; start:  8

...

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)

Section  4315 ; start:  0  ; stop:  2560
Section  4316 ; start:  1280  ; stop:  3840
Section  4317 ; start:  2560  ; stop:  5120
Section  4318 ; start:  3840  ; stop:  6400
Section  4319 ; start:  5120  ; stop:  7680
Section  4320 ; start:  6400  ; stop:  8960
Section  4321 ; start:  7680  ; stop:  10240
Section  4322 ; s

C:\Users\Calvin\AppData\Local\Temp\ipykernel_19436\4000768790.py:14: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_path, preload=True)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)

Section  5034 ; start:  0  ; stop:  2560
Section  5035 ; start:  1280  ; stop:  3840
Section  5036 ; start:  2560  ; stop:  5120
Section  5037 ; start:  3840  ; stop:  6400
Section  5038 ; start:  5120  ; stop:  7680
Section  5039 ; start:  6400  ; stop:  8960
Section  5040 ; start:  7680  ; stop:  10240
Section  5041 ; s

C:\Users\Calvin\AppData\Local\Temp\ipykernel_19436\4000768790.py:14: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_path, preload=True)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)

Section  5753 ; start:  0  ; stop:  2560
Section  5754 ; start:  1280  ; stop:  3840
Section  5755 ; start:  2560  ; stop:  5120
Section  5756 ; start:  3840  ; stop:  6400
Section  5757 ; start:  5120  ; stop:  7680
Section  5758 ; start:  6400  ; stop:  8960
Section  5759 ; start:  7680  ; stop:  10240
Section  5760 ; s

In [ ]:
np.array(labels).shape

(2876, 277)

### Save to npy

In [47]:
# np.savetxt("chb01_10i_5o_wlabel.txt", np.array(labels), delimiter=" ")
np.save("seizures_labeled", np.array(labels))

### Load npy

In [6]:
labels = np.load("seizures_labeled.npy")


### Separate features and labels and normalize features

In [16]:
from sklearn import preprocessing

x, y = labels[:,:276], labels[:,276]

x = preprocessing.normalize(x)
print(x.shape)

unique, counts = np.unique(y, return_counts=True)
print(dict(zip(unique, counts)))

(6216, 276)
{0.0: 6116, 1.0: 100}


### Separate in train and test 

In [57]:
from sklearn.model_selection import train_test_split

x_train_unb, x_test, y_train_unb, y_test =  train_test_split(x, y, test_size = 0.5, random_state = 0)
print(x_train_unb.shape)
print(y_train_unb.shape)
print(x_test.shape)
print(y_test.shape)

unique, counts = np.unique(y_train_unb, return_counts=True)
print(dict(zip(unique, counts)))

unique, counts = np.unique(y_test, return_counts=True)
print(dict(zip(unique, counts)))


(3108, 276)
(3108,)
(3108, 276)
(3108,)
{0.0: 3051, 1.0: 57}
{0.0: 3065, 1.0: 43}


### Balancing training set

In [58]:
from sklearn.utils import resample

labels_train = np.hstack((x_train_unb, np.expand_dims(y_train_unb, axis=1)))
print(labels_train.shape)

non_seizure_idx = np.nonzero(labels_train[:,276] == 0)
labels_majority = labels_train[non_seizure_idx]

seizure_idx = np.nonzero(labels_train[:,276] == 1)
labels_minority = labels_train[seizure_idx]

print("Before balancing:")
print(np.shape(labels_majority))
print(np.shape(labels_minority))

labels_minority = resample(labels_minority,
                            replace=True,
                            n_samples=int(0.5*labels_majority.shape[0]),
                            random_state=123)

print("After balancing:")
print(np.shape(labels_majority))
print(np.shape(labels_minority))

labels_balanced = np.concatenate((labels_majority, labels_minority))

np.random.shuffle(labels_balanced)
x_train, y_train = labels_balanced[:,:276], labels_balanced[:,276]

print("Full data after balancing:")
print(x_train.shape)
print(y_train.shape)

unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))
# x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size = 0.3)
# print(x_train.shape)
# print(x_test.shape)


(3108, 277)
Before balancing:
(3051, 277)
(57, 277)
After balancing:
(3051, 277)
(1525, 277)
Full data after balancing:
(4576, 276)
(4576,)
{0.0: 3051, 1.0: 1525}


### Filter features by variance and correlation 

In [42]:
from sklearn.feature_selection import VarianceThreshold

# check zero variance features
thresholder = VarianceThreshold(threshold=0)
print("Variables Kept after removing features with 0 variance: ", thresholder.fit_transform(x).shape[1])

# highly correlated features
corr = abs(pd.DataFrame(x).corr())
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
cols = [column for column in upper.columns if any(upper[column] < 0.9)]
print("Variables Kept after removing features with corr > 0.9: ", len(cols)) 

Variables Kept after removing features with 0 variance:  276
Variables Kept after removing features with corr > 0.9:  275


C:\Users\Calvin\AppData\Local\Temp\ipykernel_27256\2929964135.py:9: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))


### SVM Classifier

In [60]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.svm import SVC

svm = SVC(kernel="rbf", class_weight='balanced', random_state = 0)

# cross validation
kf = KFold(n_splits=5)
accuracy, tpr, fpr = [], [], []
for train, test in kf.split(x_train):
    svm.fit(x_train[train, :], y_train[train])
    pred = svm.predict(x_train[test])
    tn, fp, fn, tp = confusion_matrix(y_train[test], pred).ravel()
    accuracy.append((tp + tn)/(tn + fp + fn + tp))
    #print("true positive: %.4f\ntrue negative: %.4f\nfalse positive: %.4f\nfalse negative: %.4f\n" % (tp, tn, fp, fn))
    tpr.append(tp / (tp + fn))
    fpr.append(fp / (fp + tn))

print("KFold results")
print(accuracy)
print(tpr)
print(fpr)



KFold results
[0.9203056768558951, 0.9224043715846995, 0.9169398907103825, 0.9387978142076503, 0.9158469945355191]
[0.9372822299651568, 0.9663299663299664, 0.9554140127388535, 0.9404388714733543, 0.9577922077922078]
[0.08744038155802862, 0.09870550161812297, 0.10316139767054909, 0.06208053691275168, 0.10543657331136738]


In [62]:
# Testing 
svm.fit(x_train, y_train)
pred = svm.predict(x_test)
tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
accuracy = (tp + tn)/(tn + fp + fn + tp)
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print("Validation")
print("Accuracy: %.2f" % (accuracy))
print("True Positive Rate: %.2f" % (tpr))
print("False Positive Rate: %.2f" % (fpr))

Validation
Accuracy: 0.93
True Positive Rate: 0.07
False Positive Rate: 0.06


In [64]:
tp

3

In [38]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

# cross validation
kf = KFold(n_splits=5)
accuracy, tpr, fpr = [], [], []
for train, test in kf.split(x_train):
    mlp.fit(x_train[train, :], y_train[train])
    pred = mlp.predict(x_train[test])
    tn, fp, fn, tp = confusion_matrix(y_train[test], pred).ravel()
    accuracy.append((tp + tn)/(tn + fp + fn + tp))
    tpr.append(tp / (tp + fn))
    fpr.append(fp / (fp + tn))

print(accuracy)
print(tpr)
print(fpr)


c:\Users\Calvin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Calvin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Calvin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[0.9832869080779945, 0.9832869080779945, 0.9767873723305478, 0.9693593314763231, 0.9795539033457249]
[0.9583333333333334, 0.9789473684210527, 0.905982905982906, 0.8876404494382022, 0.9347826086956522]
[0.014271151885830785, 0.016293279022403257, 0.014583333333333334, 0.02327935222672065, 0.016260162601626018]


c:\Users\Calvin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [52]:
# Testing 
mlp.fit(x_train, y_train)
pred = mlp.predict(x_test)
tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
accuracy = (tp + tn)/(tn + fp + fn + tp)
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print("Validation")
print("Accuracy: %.2f" % (accuracy))
print("True Positive Rate: %.2f" % (tpr))
print("False Positive Rate: %.2f" % (fpr))

Validation
Accuracy: 0.97
True Positive Rate: 0.06
False Positive Rate: 0.02


c:\Users\Calvin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [36]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(3)

# cross validation
kf = KFold(n_splits=5)
accuracy, tpr, fpr = [], [], []
for train, test in kf.split(x_train):
    knn.fit(x_train[train, :], y_train[train])
    pred = knn.predict(x_train[test])
    tn, fp, fn, tp = confusion_matrix(y_train[test], pred).ravel()
    accuracy.append((tp + tn)/(tn + fp + fn + tp))
    tpr.append(tp / (tp + fn))
    fpr.append(fp / (fp + tn))

print(accuracy)
print(tpr)
print(fpr)


[0.9740018570102136, 0.9702878365831012, 0.9740018570102136, 0.9795728876508821, 0.974907063197026]
[0.9791666666666666, 0.9789473684210527, 0.9658119658119658, 0.9775280898876404, 0.9782608695652174]
[0.026503567787971458, 0.03054989816700611, 0.025, 0.020242914979757085, 0.02540650406504065]


In [37]:
# Testing 
knn.fit(x_train, y_train)
pred = knn.predict(x_test)
tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
accuracy = (tp + tn)/(tn + fp + fn + tp)
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print("Validation")
print("Accuracy: %.2f" % (accuracy))
print("True Positive Rate: %.2f" % (tpr))
print("False Positive Rate: %.2f" % (fpr))

Validation
Accuracy: 0.96
True Positive Rate: 0.00
False Positive Rate: 0.02
